# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("cities.csv")
print(vacation_df)


     Unnamed: 0                City  Cloud Cover Country        Date  \
0             0       Mar del Plata            0      AR  1594897332   
1             1      Cabo San Lucas            5      MX  1594903431   
2             2         Pangnirtung           20      CA  1594879318   
3             3              Albany           97      US  1594891965   
4             4             Kavieng           85      PG  1594929789   
..          ...                 ...          ...     ...         ...   
558         558          Champerico           40      GT  1594900028   
559         559             Moundou          100      TD  1594874669   
560         560            Khorinsk            3      RU  1594932026   
561         561            Yamethin          100      MM  1594940660   
562         562  Comodoro Rivadavia           90      AR  1594901094   

     Humidity  Latitude  Longitude  Maximum Temperature  Wind Speed (MPH)  
0          50    -38.00     -57.56               55.994    

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key= g_key)

In [4]:
locations = vacation_df[["Latitude", "Longitude"]]
weights = vacation_df["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
filtered_cities = vacation_df.loc[(vacation_df["Maximum Temperature"] > 70) & (vacation_df["Maximum Temperature"] < 80) & (vacation_df["Cloud Cover"] ==0) & (vacation_df["Wind Speed (MPH)"] < 10), :]
filtered_cities = filtered_cities.dropna(how="any")
filtered_cities.reset_index(inplace=True)
filtered_cities.head()

,index,Unnamed: 0,City,Cloud Cover,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed (MPH)
0,49,49,Sidi Ali,0,DZ,1594875214,49,36.10,0.42,77.846,5.43
1,50,50,Atuona,0,PF,1594913628,76,-9.80,-139.03,79.988,9.22
2,130,130,Avarua,0,CK,1594919750,73,-21.21,-159.78,71.600,4.10
3,135,135,Fasa,0,IR,1594949868,38,28.94,53.65,77.000,2.10
4,201,201,Japura,0,ID,1594941209,97,-0.32,102.35,73.814,0.50


In [6]:
filtered_cities.count()

index                  23
Unnamed: 0             23
City                   23
Cloud Cover            23
Country                23
Date                   23
Humidity               23
Latitude               23
Longitude              23
Maximum Temperature    23
Wind Speed (MPH)       23
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotels = []

for i in range(len(filtered_cities)):
    lat = filtered_cities.loc[i]["Latitude"]
    long = filtered_cities.loc[i]["Longitude"]
    
    params = {
        "location": f"{lat},{long}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    request_json = request.json()
    
    try: 
        hotels.append(request_json["results"][0]["name"])
    except:
        hotels.append(" ")
filtered_cities["Hotel Name"] = hotels
hotel_df = filtered_cities.dropna(how="any")
hotel_df.head()

,index,Unnamed: 0,City,Cloud Cover,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed (MPH),Hotel Name
0,49,49,Sidi Ali,0,DZ,1594875214,49,36.10,0.42,77.846,5.43,Sidi Ali
1,50,50,Atuona,0,PF,1594913628,76,-9.80,-139.03,79.988,9.22,Atuona
2,130,130,Avarua,0,CK,1594919750,73,-21.21,-159.78,71.600,4.10,Avarua District
3,135,135,Fasa,0,IR,1594949868,38,28.94,53.65,77.000,2.10,Fasa
4,201,201,Japura,0,ID,1594941209,97,-0.32,102.35,73.814,0.50,Djapoera


In [ ]:
hotel_df.rename(columns = {"Latitude":"Lat", "Longitude":"Lng"})

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig